# Regression
## Platform: Spark, colab.research.google.com

In [0]:
# Colab preinstalled packages
import pandas as pd
import time

In [0]:
# install Spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
# init Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [5]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
flights_dfs = spark.read.csv("/content/gdrive/My Drive/Colab Notebooks/SparkAzureTutorial/data/flights.csv", header=True, inferSchema=True)

In [7]:
flights_dfs.describe().show(5, False)

+-------+------------------+-----------------+-------+------------------+------------------+------------------+------------------+
|summary|DayofMonth        |DayOfWeek        |Carrier|OriginAirportID   |DestAirportID     |DepDelay          |ArrDelay          |
+-------+------------------+-----------------+-------+------------------+------------------+------------------+------------------+
|count  |2702218           |2702218          |2702218|2702218           |2702218           |2702218           |2702218           |
|mean   |15.797897875004903|3.899480352806472|null   |12742.597593162358|12743.000197985506|10.510732294729737|6.6550108096386005|
|stddev |8.7988350691642   |1.985924603367557|null   |1501.8408475102513|1501.8014309297723|36.02975608466093 |38.547584236791245|
|min    |1                 |1                |9E     |10140             |10140             |-63               |-94               |
|max    |31                |7                |YV     |15376             |15376     

In [8]:
data = flights_dfs.select("DayOfMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay", "ArrDelay")
data.show(5)

+----------+---------+---------------+-------------+--------+--------+
|DayOfMonth|DayOfWeek|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+---------------+-------------+--------+--------+
|        19|        5|          11433|        13303|      -3|       1|
|        19|        5|          14869|        12478|       0|      -8|
|        19|        5|          14057|        14869|      -4|     -15|
|        19|        5|          15016|        11433|      28|      24|
|        19|        5|          11193|        12892|      -6|     -11|
+----------+---------+---------------+-------------+--------+--------+
only showing top 5 rows



In [9]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]
print("Train len: {}, test len: {}".format(train.count(), test.count()))

Train len: 1891958, test len: 810260


In [10]:
assembler = VectorAssembler(inputCols = ["DayOfMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay"], outputCol="features")
training = assembler.transform(train).select(col("features"), col("ArrDelay").alias("label"))
training.show(5, truncate=False)

+------------------------------+-----+
|features                      |label|
+------------------------------+-----+
|[1.0,1.0,10140.0,10397.0,-4.0]|-11  |
|[1.0,1.0,10140.0,10397.0,-2.0]|-18  |
|[1.0,1.0,10140.0,10397.0,-2.0]|-17  |
|[1.0,1.0,10140.0,10821.0,4.0] |4    |
|[1.0,1.0,10140.0,10821.0,8.0] |-9   |
+------------------------------+-----+
only showing top 5 rows



In [11]:
testing = assembler.transform(test).select(col("features"), col("ArrDelay").alias("label"))
testing.show(5, truncate=False)

+------------------------------+-----+
|features                      |label|
+------------------------------+-----+
|[1.0,1.0,10140.0,10397.0,0.0] |-12  |
|[1.0,1.0,10140.0,10397.0,0.0] |-9   |
|[1.0,1.0,10140.0,11292.0,-8.0]|-6   |
|[1.0,1.0,10140.0,11292.0,-2.0]|-5   |
|[1.0,1.0,10140.0,11292.0,2.0] |-1   |
+------------------------------+-----+
only showing top 5 rows



In [14]:
model = LinearRegression(labelCol="label", featuresCol="features", maxIter=10, regParam=0.3)
print("Training model...")
start_time = time.time()
model = model.fit(training)
print("... training finished in {} seconds ---".format(time.time() - start_time))

Training model...
... training finished in 43.0670280456543 seconds ---


In [15]:
prediction = model.transform(testing)
predicted = prediction.select("features", "label", "prediction")
predicted.show(5, truncate=False)

+------------------------------+-----+-------------------+
|features                      |label|prediction         |
+------------------------------+-----+-------------------+
|[1.0,1.0,10140.0,10397.0,0.0] |-12  |-3.5583082357840894|
|[1.0,1.0,10140.0,10397.0,0.0] |-9   |-3.5583082357840894|
|[1.0,1.0,10140.0,11292.0,-8.0]|-6   |-11.749332091922902|
|[1.0,1.0,10140.0,11292.0,-2.0]|-5   |-5.761708393183558 |
|[1.0,1.0,10140.0,11292.0,2.0] |-1   |-1.7699592606906616|
+------------------------------+-----+-------------------+
only showing top 5 rows



In [16]:
trainingSummary = model.summary
rmse = trainingSummary.rootMeanSquaredError
r2 = trainingSummary.r2
print("RMSE: {}".format(rmse))
print("R2: {}".format(r2))

RMSE: 13.216181590888775
R2: 0.8825939547575687


## Platform: Pandas, scikit-learn, colab.research.google.com

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [18]:
flights_df = flights_dfs.toPandas()
flights_df.describe()

,DayofMonth,DayOfWeek,OriginAirportID,DestAirportID,DepDelay,ArrDelay
count,2.702218e+06,2.702218e+06,2.702218e+06,2.702218e+06,2.702218e+06,2.702218e+06
mean,1.579790e+01,3.899480e+00,1.274260e+04,1.274300e+04,1.051073e+01,6.655011e+00
std,8.798835e+00,1.985925e+00,1.501841e+03,1.501801e+03,3.602976e+01,3.854758e+01
min,1.000000e+00,1.000000e+00,1.014000e+04,1.014000e+04,-6.300000e+01,-9.400000e+01
25%,8.000000e+00,2.000000e+00,1.129200e+04,1.129200e+04,-4.000000e+00,-1.100000e+01
50%,1.600000e+01,4.000000e+00,1.289200e+04,1.289200e+04,-1.000000e+00,-3.000000e+00
75%,2.300000e+01,6.000000e+00,1.405700e+04,1.405700e+04,9.000000e+00,1.000000e+01
max,3.100000e+01,7.000000e+00,1.537600e+04,1.537600e+04,1.863000e+03,1.845000e+03


In [0]:
X = flights_df.loc[:, ["DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay"]]
y = flights_df.loc[:, "ArrDelay"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [20]:
print("Training model...")
start_time = time.time()
model = LinearRegression().fit(X_train, y_train)
print("... training finished in {} seconds ---".format(time.time() - start_time))
y_pred = model.predict(X_test)

Training model...
... training finished in 0.35336780548095703 seconds ---


In [21]:
predicted = pd.DataFrame(X_test)
predicted = pd.concat([predicted, 
                       pd.DataFrame(y_test, index=y_test.index), 
                       pd.DataFrame(y_pred, index=y_test.index)], 
                      axis=1, ignore_index=True)
predicted.columns = list(X.columns) + ["label"] + ["y_pred"]
predicted.head(5)

,DayofMonth,DayOfWeek,OriginAirportID,DestAirportID,DepDelay,label,y_pred
2159824,6,5,10821,11292,10,-17,5.834453
632083,16,4,15304,14843,4,-8,0.132481
115502,27,6,10821,13303,-8,-21,-12.630798
2577846,2,3,14492,15304,-2,-11,-6.182267
1194206,8,1,12266,14122,14,11,10.058543


In [22]:
rmse = mean_squared_error(y_test, y_pred) ** 0.5
r2 = r2_score(y_test, y_pred)
assert r2 == model.score(X_test, y_test)
print("RMSE: {}".format(rmse))
print("R2: {}".format(r2))

RMSE: 13.160356997190407
R2: 0.8836323895450338


### Reusing Spark split data to compare metrics

In [0]:
train_df = train.toPandas()
test_df = test.toPandas()

In [0]:
X_train = train_df.drop("ArrDelay", axis=1)
y_train = train_df["ArrDelay"]
X_test = test_df.drop("ArrDelay", axis=1)
y_test = test_df["ArrDelay"]

In [25]:
print("Training model...")
start_time = time.time()
model = LinearRegression().fit(X_train, y_train)
print("... training finished in {} seconds ---".format(time.time() - start_time))
y_pred = model.predict(X_test)

Training model...
... training finished in 0.31351709365844727 seconds ---


In [26]:
predicted = pd.DataFrame(X_test)
predicted = pd.concat([predicted, 
                       pd.DataFrame(y_test, index=y_test.index), 
                       pd.DataFrame(y_pred, index=y_test.index)], 
                      axis=1, ignore_index=True)
predicted.columns = list(X.columns) + ["label"] + ["y_pred"]
predicted.head(5)

,DayofMonth,DayOfWeek,OriginAirportID,DestAirportID,DepDelay,label,y_pred
0,1,1,10140,10397,0,-12,-3.644583
1,1,1,10140,10397,0,-9,-3.644583
2,1,1,10140,11292,-8,-6,-11.900143
3,1,1,10140,11292,-2,-5,-5.865919
4,1,1,10140,11292,2,-1,-1.843104


In [27]:
rmse = mean_squared_error(y_test, y_pred) ** 0.5
r2 = r2_score(y_test, y_pred)
assert r2 == model.score(X_test, y_test)
print("RMSE: {}".format(rmse))
print("R2: {}".format(r2))

RMSE: 13.232373510294558
R2: 0.8818278251240874
